In [5]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from src.benchmarkers import *
from src.benchmarkersV2 import *
import time
from tqdm import tqdm
from matplotlib import pyplot as plt
np.random.seed(0)

In [14]:
#def print_name(name):
#    print(name)
#
#with h5py.File(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\32\ds_10_core.h5", "r") as file:
#    file.visit(print_name)
#    image1 = np.array(file["example_0/image_feature/image1/boundingbox_feature/bb1/image_1_feature/image"][:])
#
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\64\ds_10.parquet")
#image2 = np.array(table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature").values.field("image").to_pylist())
#image2np = np.frombuffer(image2[0], dtype=np.float64).reshape(3, 32, 32)
#image1 == image2np

In [21]:
N = [10,100,200,300,500]
dimensions = [32,64,125,192]
selected_label = 10
iterations = 100

In [3]:
generator = Generator()

### Generation

In [10]:
for item in tqdm(N):
    for dim in dimensions:
        generator.create_dataset(item,f"outputs/v2/{dim}/ds_{item}",dim)
        generator.create_arrow_file(f"outputs/v2/{dim}/ds_{item}")
        generator.create_arrow_stream(f"outputs/v2/{dim}/ds_{item}")

100%|██████████| 5/5 [07:53<00:00, 94.61s/it] 
